# Rusty Bargain

# 1. Contenido

* [1 Contenido](#content)
* [2 Introducción](#intro)
* [3 Inicialización](#inic)
    * [3.1 Cargar Librerias](#library)
    * [3.2 Cargar Datos](#datos)
* [4 Exploración y preparación los datos](#exp) 
    * [4.1 Preprocesamiento](#prep)
         * [4.1.1 Valores Ausentes](#nan)
* [5 Entrenamiento del modelo](#train)
    * [5.1 Segmentación datos entrenamiento y validación](#seg)
    * [5.2 Segmentación datos selección hiperparametros](#seg2)
    * [5.3 Regresión lineal](#rl)
    * [5.4 Bosque aleatorio](#rf)
    * [5.5 LightGBM](#lgbm)
    * [5.6 CatBoost](#cb)
    * [5.7 XGBoost](#xgb)
* [6 Analisis del Modelo](#analisis)

## 2. Introducción

En el presente proyecto procederemos a desarrollar un de modelo de machine learning para la empresa Rusty Bargain con el objetivo de predecir el valor de mercado de venta de coches de segunda mano a partir del historial, especificaciones técnicas, versiones de equipamiento y precios de los vehículos vendidos por la empresa. El modelo ayudará en el desarrollo de una app por parte de la empresa para atraer nuevos clientes.

Comenzaremos revisando los datos proporcionados, realizaremos un análisis de los datos y desarrollaremos el modelo solicitado

## 3. Inicialización

Se procede a cargar las librerías que se utilizaran en el proyecto.

### 3.1 Cargar librerias

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OrdinalEncoder

from sklearn.model_selection import GridSearchCV
import numpy as np
import xgboost as xgb
import lightgbm as lgbm
from sklearn.metrics import mean_squared_error

import time

### 3.2 Cargar datos

Se procede a cargar los datos.

In [2]:
data = pd.read_csv('datasets/car_data.csv')

## 4. Exploración y preparación los datos

A continuación, procederemos a realizar una exploración sobre las bases de datos entregadas por Rusty Bargain.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [4]:
data.head(5)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


Observando la base de datos podemos apreciar que existen valores ausentes en 5 de las columnas de la base de datos, "VehicleType", "Gearbox", "Model", "FuelType" y "NotRepaired", siendo en esta última columna donde se encuentra una mayor cantidad de datos ausentes correspondiendo a alrededor de un 21% de los datos. Por otra parte, en los tipos de datos podemos apreciar que las columnas "DateCrawled", "DateCreated" y "LastSeen" son de tipo object cuando deberian ser de tipo datetime.

### 4.1 Preprocesamiento

Comenzaremos cambiando el tipo de datos de las columnas correspondientes a fechas.

In [5]:
data['DateCrawled'] = pd.to_datetime(data['DateCrawled'], format='%d/%m/%Y %H:%M')
data['DateCreated'] = pd.to_datetime(data['DateCreated'], format='%d/%m/%Y %H:%M')
data['LastSeen'] = pd.to_datetime(data['LastSeen'], format='%d/%m/%Y %H:%M')

Continuaremos observando si existen filas duplicadas y eliminandolas en el caso de que existan.

In [6]:
data.duplicated().sum()

262

In [7]:
data = data.drop_duplicates()

Procederemos observando si existen valores extraños en las columnas.

In [8]:
for column in data[['Price', 'RegistrationYear', 'Power', 'Mileage', 'RegistrationMonth', 'NumberOfPictures']]:
    print(f'{column}: {(data[data[column] < 0][column]).count()}')

Price: 0
RegistrationYear: 0
Power: 0
Mileage: 0
RegistrationMonth: 0
NumberOfPictures: 0


In [9]:
for column in data[['VehicleType', 'Gearbox', 'Model', 'FuelType','Brand', 'NotRepaired']]:
    print(f'{column}: {data[column].unique()}')

VehicleType: [nan 'coupe' 'suv' 'small' 'sedan' 'convertible' 'bus' 'wagon' 'other']
Gearbox: ['manual' 'auto' nan]
Model: ['golf' nan 'grand' 'fabia' '3er' '2_reihe' 'other' 'c_max' '3_reihe'
 'passat' 'navara' 'ka' 'polo' 'twingo' 'a_klasse' 'scirocco' '5er'
 'meriva' 'arosa' 'c4' 'civic' 'transporter' 'punto' 'e_klasse' 'clio'
 'kadett' 'kangoo' 'corsa' 'one' 'fortwo' '1er' 'b_klasse' 'signum'
 'astra' 'a8' 'jetta' 'fiesta' 'c_klasse' 'micra' 'vito' 'sprinter' '156'
 'escort' 'forester' 'xc_reihe' 'scenic' 'a4' 'a1' 'insignia' 'combo'
 'focus' 'tt' 'a6' 'jazz' 'omega' 'slk' '7er' '80' '147' '100' 'z_reihe'
 'sportage' 'sorento' 'v40' 'ibiza' 'mustang' 'eos' 'touran' 'getz' 'a3'
 'almera' 'megane' 'lupo' 'r19' 'zafira' 'caddy' 'mondeo' 'cordoba' 'colt'
 'impreza' 'vectra' 'berlingo' 'tiguan' 'i_reihe' 'espace' 'sharan'
 '6_reihe' 'panda' 'up' 'seicento' 'ceed' '5_reihe' 'yeti' 'octavia' 'mii'
 'rx_reihe' '6er' 'modus' 'fox' 'matiz' 'beetle' 'c1' 'rio' 'touareg'
 'logan' 'spider' 'cuo

No se aprecian valores extraños en las columnas.

#### 4.1.1. Valores Ausentes

Debido a que las columnas con valores ausentes no pueden ser rellenadas con respecto a columnas que tengan sus datos completos, las columnas más importantes para poder definir las características de los autos serian el modelo, tipo de vehículo y la marca, pero dos de estas columnas poseen valores ausentes y que en el caso de la columna con mayor cantidad de valores ausentes, “NotRepaired”, no hay forma de rellenarla sin datos adicionales que no poseemos, procederemos a eliminar las filas con valores ausentes.

In [10]:
data = data.dropna()

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 245567 entries, 3 to 354367
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   DateCrawled        245567 non-null  datetime64[ns]
 1   Price              245567 non-null  int64         
 2   VehicleType        245567 non-null  object        
 3   RegistrationYear   245567 non-null  int64         
 4   Gearbox            245567 non-null  object        
 5   Power              245567 non-null  int64         
 6   Model              245567 non-null  object        
 7   Mileage            245567 non-null  int64         
 8   RegistrationMonth  245567 non-null  int64         
 9   FuelType           245567 non-null  object        
 10  Brand              245567 non-null  object        
 11  NotRepaired        245567 non-null  object        
 12  DateCreated        245567 non-null  datetime64[ns]
 13  NumberOfPictures   245567 non-null  int64   

## 5. Entrenamiento del modelo 

Procederemos a realizar la segmentación de los datos, en este caso utilizaremos 2 segmentaciones distintas para los diferentes modelos a entrenar. La primera es la segmentación de los datos para el entrenamiento y validación, mientras la segunda será una segmentación a partir de un subconjunto aleatorio de los datos para poder decidir los mejores hiperparametros para los modelos disminuyendo el tiempo de proceso. 

### 5.1 Segmentación datos entrenamiento y validación

In [14]:
encoder = OrdinalEncoder()
data_ordinal = pd.DataFrame(encoder.fit_transform(data), columns=data.columns)
features_train, features_valid, target_train, target_valid = train_test_split(
    data_ordinal.drop(['DateCrawled','Price','DateCreated','LastSeen'], axis=1), 
    data_ordinal.Price, test_size=0.25, random_state=12345)

### 5.2 Segmentación datos selección hiperparametros

In [15]:
data_sample = data_ordinal.sample(frac = 0.01, replace = False, random_state = 12345)

In [16]:
features_train_sample, features_valid_sample, target_train_sample, target_valid_sample = train_test_split(
    data_sample.drop(['DateCrawled','Price','DateCreated','LastSeen'], axis=1), 
    data_sample.Price, test_size=0.25, random_state=12345)

### 5.3 Regresión lineal

Comenzaremos con un modelo de regresión lineal para utilizarlo como prueba de cordura con respecto al resto de los modelos.

In [21]:
modelLR = LinearRegression()
start_train = time.time()
modelLR.fit(features_train, target_train)
stop_train = time.time()
start_predict = time.time()
predictionsLR = modelLR.predict(features_valid) 
resultLR = np.sqrt(mean_squared_error(target_valid, predictionsLR))
stop_predict = time.time()
print('RMSE:', resultLR, 'Time Train:', round(stop_train - start_train,5), 'seconds', 'Time Predict:', round(stop_predict - start_predict,5), 'seconds')

RMSE: 543.0050966141498 Time Train: 0.14062 seconds Time Predict: 0.00499 seconds


### 5.4 Bosque aleatorio

Continuaremos con un modelo de bosque aleatorio.

In [38]:
parameters = {
    "criterion": ("squared_error", "absolute_error","poisson"),
    "max_depth": np.linspace(1, 10, 5).astype("int"),
    "n_estimators": np.linspace(50, 200, 20).astype("int")
}

gridSearchModel = GridSearchCV(
    estimator=RandomForestRegressor(random_state=12345),
    param_grid=parameters,
    scoring="neg_mean_squared_error",
    n_jobs=-1
)
gridSearchModel.fit(features_train_sample, target_train_sample)
gridSearchModel.best_params_

{'criterion': 'absolute_error', 'max_depth': 10, 'n_estimators': 160}

In [39]:
modelRF = RandomForestRegressor(random_state=12345, n_estimators=160, criterion='absolute_error', max_depth=10)
start_train = time.time()
modelRF.fit(features_train, target_train) 
stop_train = time.time()
start_predict = time.time()
predictionsRF = modelRF.predict(features_valid) 
resultRF = np.sqrt(mean_squared_error(target_valid, predictionsRF))
stop_predict = time.time()
print('RMSE:', resultRF, 'Time Train:', round(stop_train - start_train,5), 'seconds', 'Time Predict:', round(stop_predict - start_predict,5), 'seconds')

RMSE: 359.7298274442907 Time Train: 48165.32877 seconds Time Predict: 1.24055 seconds


### 5.5. LightGBM

Procederemos a entrenar un modelo de potenciación de gradiente en base a lightGBM.

In [22]:
LGB = lgbm.LGBMRegressor()

In [25]:
parameters = {'boosting_type' : ("gbdt", "goss"),
              'max_depth' : np.linspace(1, 10, 5).astype("int"),
              'num_leaves' : np.linspace(1, 15, 5).astype("int"),
              'learning_rate' : [0.01, 0.05, 0.1, 0.3, 0.5],
              'objective' : ['root_mean_squared_error'],
             'metric' : ['rmse'],
             'error_score' : ['raise']}

In [26]:
Grid_LGB = GridSearchCV(estimator=lgbm.LGBMRegressor(random_state=12345,force_row_wise='true'), param_grid = parameters, scoring="neg_mean_squared_error", n_jobs=-1)
Grid_LGB.fit(features_train_sample, target_train_sample)
Grid_LGB.best_params_

{'boosting_type': 'gbdt',
 'error_score': 'raise',
 'learning_rate': 0.3,
 'max_depth': 3,
 'metric': 'rmse',
 'num_leaves': 4,
 'objective': 'root_mean_squared_error'}

In [31]:
lgb_model = lgbm.LGBMRegressor(boosting_type='gbdt',
    objective='root_mean_squared_error',
    max_depth=3,
    num_leaves=4,
    learning_rate=0.3,
    metric='rmse',
    force_row_wise='true',
    random_state=12345)
start_train = time.time()
lgb_model.fit(features_train, target_train) 
stop_train = time.time()
start_predict = time.time()
preds_lgb_model = lgb_model.predict(features_valid)
resultLGB = np.sqrt(mean_squared_error(target_valid, preds_lgb_model))
stop_predict = time.time()
print('RMSE:', resultLGB, 'Time Train:', round(stop_train - start_train,5), 'seconds', 'Time Predict:', round(stop_predict - start_predict,5), 'seconds')

RMSE: 347.03707672261646 Time Train: 0.50166 seconds Time Predict: 0.08178 seconds


### 5.6 CatBoost

Continuaremos con otro modelo de potenciación de gradiente, en este caso CatBoost.

In [28]:
CBR = CatBoostRegressor()

In [29]:
parameters = {'depth'         : [4,5,6,7,8,9, 10],
                 'learning_rate' : [0.01,0.02,0.03,0.04],
                  'iterations'    : [10, 20,30,40,50,60,70,80,90,100]}

In [30]:
Grid_CBR = GridSearchCV(estimator=CBR, param_grid = parameters, scoring="neg_mean_squared_error", n_jobs=-1)
Grid_CBR.fit(features_train_sample, target_train_sample)
Grid_CBR.best_params_

0:	learn: 868.7195459	total: 150ms	remaining: 14.8s
1:	learn: 849.4274725	total: 152ms	remaining: 7.46s
2:	learn: 829.1774504	total: 155ms	remaining: 5.01s
3:	learn: 810.1722163	total: 157ms	remaining: 3.78s
4:	learn: 791.2515407	total: 160ms	remaining: 3.04s
5:	learn: 772.0029161	total: 161ms	remaining: 2.53s
6:	learn: 754.5624962	total: 165ms	remaining: 2.19s
7:	learn: 736.5941127	total: 168ms	remaining: 1.93s
8:	learn: 721.8655990	total: 171ms	remaining: 1.73s
9:	learn: 706.4785863	total: 174ms	remaining: 1.56s
10:	learn: 691.1479731	total: 176ms	remaining: 1.43s
11:	learn: 676.7951202	total: 179ms	remaining: 1.31s
12:	learn: 664.4325984	total: 182ms	remaining: 1.22s
13:	learn: 651.1839418	total: 185ms	remaining: 1.13s
14:	learn: 637.9711595	total: 187ms	remaining: 1.06s
15:	learn: 626.6229849	total: 189ms	remaining: 995ms
16:	learn: 614.9927001	total: 192ms	remaining: 937ms
17:	learn: 604.4870543	total: 194ms	remaining: 883ms
18:	learn: 594.4978029	total: 196ms	remaining: 835ms
19:

{'depth': 7, 'iterations': 100, 'learning_rate': 0.04}

In [32]:
catf = []
modelCB = CatBoostRegressor(depth=7, learning_rate=0.04, loss_function="RMSE", iterations=100, random_seed=12345)
start_train = time.time()
modelCB.fit(features_train, target_train, cat_features=catf)
stop_train = time.time()
start_predict = time.time()
probabilitiesCB = modelCB.predict(features_valid)
resultCB = np.sqrt(mean_squared_error(target_valid, probabilitiesCB))
stop_predict = time.time()
print('RMSE:', resultCB, 'Time Train:', round(stop_train - start_train,5), 'seconds', 'Time Predict:', round(stop_predict - start_predict,5), 'seconds')

0:	learn: 863.1361186	total: 29.6ms	remaining: 2.93s
1:	learn: 839.1469777	total: 47.4ms	remaining: 2.32s
2:	learn: 816.3581373	total: 72.7ms	remaining: 2.35s
3:	learn: 794.8876101	total: 97.4ms	remaining: 2.34s
4:	learn: 774.0385318	total: 119ms	remaining: 2.26s
5:	learn: 754.2746685	total: 144ms	remaining: 2.25s
6:	learn: 735.2939241	total: 172ms	remaining: 2.28s
7:	learn: 717.3741630	total: 197ms	remaining: 2.27s
8:	learn: 700.2604705	total: 217ms	remaining: 2.19s
9:	learn: 684.0986433	total: 251ms	remaining: 2.26s
10:	learn: 668.9080166	total: 276ms	remaining: 2.23s
11:	learn: 654.1981913	total: 302ms	remaining: 2.21s
12:	learn: 640.1810009	total: 323ms	remaining: 2.16s
13:	learn: 627.0282724	total: 345ms	remaining: 2.12s
14:	learn: 614.3748040	total: 370ms	remaining: 2.1s
15:	learn: 602.4968239	total: 393ms	remaining: 2.06s
16:	learn: 590.7470639	total: 413ms	remaining: 2.02s
17:	learn: 579.9802312	total: 439ms	remaining: 2s
18:	learn: 569.6601808	total: 479ms	remaining: 2.04s
19:

### 5.7 XGBoost

Concluiremos con un último modelo de potenciación de gradiente, XGBoost.

In [45]:
xgb_model = xgb.XGBRegressor() 
xgb_params = {'max_depth': [1,2,3,4,5,6,7,8,9,10], 'eta': [0,0.2,0.4,0.6,0.8,1]}

In [46]:
clf = GridSearchCV(estimator=xgb_model, param_grid =xgb_params, scoring='neg_mean_squared_error',  n_jobs=-1)
clf.fit(features_train_sample, target_train_sample)
clf.best_params_

{'eta': 0.2, 'max_depth': 3}

In [44]:
xgb_model = xgb.XGBRegressor(eta=0.2,
    max_depth=3,
    random_state=12345)
start_train = time.time()
xgb_model.fit(features_train, target_train) 
stop_train = time.time()
start_predict = time.time()
preds_xgb_model = xgb_model.predict(features_valid)
resultXGB = np.sqrt(mean_squared_error(target_valid, preds_xgb_model))
stop_predict = time.time()
print('RMSE:', resultXGB, 'Time Train:', round(stop_train - start_train,5), 'seconds', 'Time Predict:', round(stop_predict - start_predict,5), 'seconds')

RMSE: 340.6366895646541 Time Train: 3.35101 seconds Time Predict: 0.04488 seconds


## Análisis del modelo

Al comparar los distintos modelos creados, utilizando como prueba de cordura el modelo en base a la regresión lineal que posee un RMSE de 543, un tiempo de entrenamiento de 0,14 segundos y un tiempo de predicción de 0,005 segundos, en general todos los modelos basados en potenciación de gradiente cumplen con un mejor error cuadrático medio que nuestra prueba de cordura, por otra parte con respecto al tiempo de entrenamiento y predicción son mayores que en el modelo de regresión lineal pero en pequeñas cantidades. 

De los tres modelos de potenciación de gradiente el que mejor desempeño tiene con respecto al RMSE es el basado en XGBoost con un resultado de 340, con respecto al tiempo de entrenamiento el mejor resultado fue con el modelo en base a LightGBM con un resultado de 0,5 segundos, finalmente para el tiempo de predicción el mejor resultado lo posee el modelo en base a CatBoost con un resultado de 0,03 segundos. Entre los tres modelos el que posee los mejores resultados ponderados es el LightGBM teniendo el mejor tiempo de entrenamiento, un sexto de tiempo del siguiente modelo, solo posee 7 puntos de diferencia en el RMSE con respecto al modelo XGBoost y para el tiempo de predicción es el doble del modelo CatBoost pero sigue siendo un valor mínimo de 0,08 segundos.

 Finalmente es de considerar que el modelo basado en ramdon forest posee un buen valor de EMSE en comparación con los modelos de potenciación de gradiente pero el tiempo de entrenamiento asciende a magnitudes mucho mayores que el resto de los modelos llegando en este caso a demorarse alrededor de 13 horas por lo que es descartado inmediatamente.
